# Análise de sentimentos em tweets feito por usuários do Twitter

       Para este projeto, sera proposto uma forma de realizar análise de sentimentos em tweets feitos por usúarios do Twitter (em inglês), seja um tweet positivo, negativo ou neutro. Para isso, utilizaremos a linguagem de programação Python com o auxilio da biblioteca nltk que possui um corpus disponível para uso de 30.000 tweets, sendo dividos entre 5.000 tweets positivos, 5.000 tweets negativos e 20.000 tweets neutros.

## Leitura da base de dados

        A biblioteca nltk fornece um corpus livre para uso. Para utilizar ele primeiramente é necessário instalar a base de dados e o punkt que realiza a tokenização da mesma, via os comandos:

In [ ]:
import nltk
nltk.download('twitter_samples')
nltk.download('punkt')

Uma vez com o download concluído, podemos realizar a leitura da base de dados e exibir no console um exemplo de tweet após a tokenização:

In [2]:
from nltk.corpus import twitter_samples
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
neutral_tweets = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(tweet_tokens[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


## Normalização dos dados

    Palavras possuem diferentes formas, por exemplo: o verbo "write" em inglés pode aparecer nas formas "wrote" e "writing". Dependendo da análise, é necessário normalizar estas palavras para que elas não sejam tratadas como diferentes pelo modelo, como será feito neste projeto. Em NLP, normalização dos dados é o processo de transformar uma palavra para sua forma canônica, ou seja, sempre que aparecer "wrote" ou "writing" ele será transformado para "write". Para este projeto, iremos realizar a normalização dos dados através da técnica de lemmatization (Lematização), para isso será necessário determinar o contexto de cada palavra - em outras palavras, qual a tag de cada palavra, utilizando um postagger:

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [5]:
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


Exibindo o mesmo tweet exemplo do que da última vez, podemos ver que para cada palavra (token) uma tag foi associado – por exemplo, o verbo "being" foi associada a tag VBG (Verbo no gerúndio ou pretérito presente). Agora, podemos prosseguir com a técnica de lemetização:

In [ ]:
import nltk
nltk.download('wordnet')

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence
print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


De forma resumida, uma palavra que possui começa com a tag 'NN' vai ser um noun (substantivo) e uma tag que começa com 'VB' vai ser um verbo. Passando a posição de cada palavra para a função de lemetização, ela nos retorna a sentença com as palavras substituidas para sua forma canônica – como no exemplo do primeiro tweet, podemos perceber que o verbo "being" se tornou "be" e o substantivo "members" se tornou "member".

## Pré processamento - Remoção de ruídos

    Como vimos no nosso tweet de exemplo, há diversos caracteres que não acrescentam nenhuma informação relevante - por exemplo: o '_' e o '@' não são interessantes para a análise de sentimentos. Além destes caracteres irrelevantes, hyperlinks e stopwords como (the,a,an, etc)  também são comuns em tweets, e nao irá agregar nada para a análise de sentimentos. Mais ainda, todos as letras serão transformados em minisculas para facilitar ainda mais o entendimento da rede:

In [ ]:
import nltk
nltk.download('stopwords')

In [10]:
import re, string
from nltk.corpus import stopwords

tweet_tokens_example = lemmatize_sentence(tweet_tokens[0])
def pre_processing(tweet_tokens, stop_words = ()):
    new_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        if len(token) >= 1 and token not in string.punctuation and token.lower() not in stop_words:
            new_tokens.append(token.lower())
    return new_tokens
stop_words = stopwords.words('english')
print(pre_processing(tweet_tokens_example, stop_words))


['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


Executando a função de pré processamento todas as letras foram transformadas em minusculas e os hyperlinks, stopwords como "in, the, my" e os '@' foram removidos.

## Treinamento do modelo e avaliação dos resultados

    Para o treinamento do modelo em python, primeiro é necessário converter os tweets tokenizados para um dicionário. Para isso, basta usarmos a função de conversão, passando por parametrôs uma lista de tokens:

In [39]:
def convert_list_to_dict(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)
        
positive_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tokens = twitter_samples.tokenized('negative_tweets.json')
neutral_tokens = twitter_samples.tokenized('tweets.20150430-223406.json')
positive_tokens_list = []
negative_tokens_list = []
neutral_tokens_list = []

stop_words = stopwords.words('english')
for tokens in positive_tokens:
    positive_tokens_list.append(pre_processing(tokens, stop_words))

for tokens in negative_tokens:
    negative_tokens_list.append(pre_processing(tokens, stop_words))

for tokens in neutral_tokens:
    neutral_tokens_list.append(pre_processing(tokens, stop_words))

positive_tokens_for_model = convert_list_to_dict(positive_tokens_list)
negative_tokens_for_model = convert_list_to_dict(negative_tokens_list)
neutral_tokens_for_model = convert_list_to_dict(neutral_tokens_list)

Executando o trecho de código acima, criamos uma lista para cada tipo de tokens: positivo, negativo e neutro e adicionamos os respectivos tokens pré processados em sua respectiva lista. Após isso, utilizamos a função convert_to_list para converter as listas criadas para um dicionário. A base de dados foi dividida em 80% (24.000) para o treinamento e 20% (6.000) para testes:

In [40]:
import random
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]
neutral_dataset = [(tweet_dict, "Neutral")
                     for tweet_dict in neutral_tokens_for_model]
dataset = positive_dataset + negative_dataset + neutral_dataset
random.shuffle(dataset)
train_data = dataset[:24000]
test_data = dataset[24000:]

Para este projeto, foi utilizado o classificador Naive Bayes, um classificador probabilístico baseado na aplicação do Teorema de Bayes, que por sua vez possui fortes premisas independentes entre as características.

In [44]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

Com duração de aproximadamente 5 minutos, o modelo foi treinado podemos ver os resultados obtidos além das 10 informações mais relevantes:

In [45]:
print("Accuracy is:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Accuracy is: 0.993
Most Informative Features
                      :( = True           Negati : Positi =   2366.9 : 1.0
                      :) = True           Positi : Negati =   1867.8 : 1.0
                      :d = True           Positi : Neutra =   1359.7 : 1.0
                     :-( = True           Negati : Neutra =    609.7 : 1.0
                      rt = True           Neutra : Negati =    588.8 : 1.0
                   david = True           Neutra : Positi =    221.9 : 1.0
                    tory = True           Neutra : Positi =    221.1 : 1.0
                     :-) = True           Positi : Neutra =    206.8 : 1.0
                   wanna = True           Negati : Neutra =    187.6 : 1.0
                      <3 = True           Positi : Neutra =    177.5 : 1.0
None


O modelo obteve uma acurácia de 99.2%. Na tabela de informações mais relavantes, cada linha informa a relação sobre o grupo de tweets em que o mesmo mais apareceu pelo grupo em que ele menos apareceu, por exemplo: o token mais comum na base de dados é o emoticon de tristeza e possui uma razão de 5660/1 tweets para o grupo neutro, ou seja, em somente 1 tweet a cada 5660 que possui o emoticon ':(', não estará relacionado a um sentimento negativo. Um ótimo resultado!

Podemos ainda realizar testes com tweets personalizados. No exemplo abaixo, hipoteticamente tweetamos um tweet com sentimento de tristeza, ou seja, negativo. Vamos ver como o nosso modelo se comporta e se ele irá acertar o resultado:

In [46]:
from nltk.tokenize import word_tokenize

custom_tweet = "I lost my dog yesterday. Feels bad! :("

custom_tokens = pre_processing(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative


Traduzido, tweetamos que nosso cachorro morreu ontem o que é um sentimento negativo. Usando esse tweet para entrada do modelo, ele nos retornou a saída corretamente. Agora vamos realizar um novo teste, mas dessa vez com um sentimento alegre:

In [47]:
from nltk.tokenize import word_tokenize

custom_tweet = "I got lucky today and I won 50$ betting with a friend! =D"

custom_tokens = pre_processing(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive


Traduzido, tweetamos que ganhamos 50 dolares em um aposta com um amigo e o modelo preveu que era um sentimento positivo. Correto! Por fim, testaremos o modelo para um sentimento neutro:

In [48]:
from nltk.tokenize import word_tokenize

custom_tweet = "I played soccer with David yesterday."

custom_tokens = pre_processing(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Neutral


Traduzindo, tweetamos que jogamos futebol com o David ontem e o modelo prevêu corretamente mais uma vez.

## Conclusão

Foi treinado um modelo capaz de realizar análise de sentimentos em tweets do idioma inglês. O modelo obteve uma boa performance para o problema em questão. Escolhi esse tema por me despertar um interesse depois que eu li um artigo que realizava essa mesma coisa, mas para criticas de filmes. Acredito que foi um tema legal de se trabalhar e eu espero que seja relevante para o projeto da disciplina.
Leonardo Santos Miranda - Mestrado